# importing libraries

In [3]:
import nltk
import tensorflow
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

59 documents
15 classes ['Food', 'Mask', 'Music', 'Sanitizer', 'cases', 'corona', 'goodbye', 'greeting', 'love', 'medicines', 'options', 'precautions', 'symptoms', 'thanks', 'vaccine']
100 unique lemmatized words ["'s", ',', 'a', 'against', 'am', 'anyone', 'are', 'available', 'avoid', 'avoidable', 'avoidance', 'awesome', 'be', 'best', 'book', 'bot', 'bye', 'can', 'case', 'chatting', 'check', 'choice', 'corona', 'could', 'covid19', 'day', 'do', 'eating', 'effective', 'favourite', 'food', 'for', 'genre', 'get', 'good', 'goodbye', 'guideline', 'hand', 'having', 'healthy', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'if', 'immunity', 'is', 'know', 'later', 'love', 'make', 'mask', 'me', 'meal', 'medicine', 'more', 'music', 'my', 'neccesary', 'neccessary', 'next', 'nice', 'number', 'of', 'offered', 'patient', 'precaution', 'prevent', 'provide', 'safe', 'sanitizer', 'santizer', 'see', 'should', 'support', 'symptom', 'tell', 'thank', 'thanks', 'that', 'the', 'there'

# creating training data

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


# training the model

In [16]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=5, verbose=1)
model.save('our_model.h5', hist)

print("model created")

Epoch 1/500
12/12 [==============================] - 0s 831us/step - loss: 2.7712 - accuracy: 0.0339
Epoch 2/500
12/12 [==============================] - 0s 1ms/step - loss: 2.6604 - accuracy: 0.0847
Epoch 3/500
12/12 [==============================] - 0s 1ms/step - loss: 2.6179 - accuracy: 0.1356
Epoch 4/500
12/12 [==============================] - 0s 1ms/step - loss: 2.5442 - accuracy: 0.1695
Epoch 5/500
12/12 [==============================] - 0s 1ms/step - loss: 2.5590 - accuracy: 0.1186
Epoch 6/500
12/12 [==============================] - 0s 915us/step - loss: 2.4442 - accuracy: 0.1695
Epoch 7/500
12/12 [==============================] - 0s 1ms/step - loss: 2.3412 - accuracy: 0.2542
Epoch 8/500
12/12 [==============================] - 0s 1ms/step - loss: 2.3159 - accuracy: 0.3390
Epoch 9/500
12/12 [==============================] - 0s 1ms/step - loss: 2.1445 - accuracy: 0.4068
Epoch 10/500
12/12 [==============================] - 0s 1ms/step - loss: 1.9884 - accuracy: 0.3898
Epoch

12/12 [==============================] - 0s 997us/step - loss: 0.0214 - accuracy: 1.0000
Epoch 163/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 1.0000
Epoch 164/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0411 - accuracy: 0.9831
Epoch 165/500
12/12 [==============================] - 0s 997us/step - loss: 0.0511 - accuracy: 0.9831
Epoch 166/500
12/12 [==============================] - 0s 748us/step - loss: 0.0775 - accuracy: 0.9831
Epoch 167/500
12/12 [==============================] - 0s 665us/step - loss: 0.0218 - accuracy: 1.0000
Epoch 168/500
12/12 [==============================] - 0s 831us/step - loss: 0.0304 - accuracy: 1.0000
Epoch 169/500
12/12 [==============================] - 0s 831us/step - loss: 0.0545 - accuracy: 0.9831
Epoch 170/500
12/12 [==============================] - 0s 914us/step - loss: 0.0117 - accuracy: 1.0000
Epoch 171/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0337 - accurac

12/12 [==============================] - 0s 997us/step - loss: 0.0244 - accuracy: 1.0000
Epoch 323/500
12/12 [==============================] - 0s 997us/step - loss: 0.0738 - accuracy: 0.9661
Epoch 324/500
12/12 [==============================] - 0s 997us/step - loss: 0.0434 - accuracy: 0.9661
Epoch 325/500
12/12 [==============================] - 0s 748us/step - loss: 0.0100 - accuracy: 1.0000
Epoch 326/500
12/12 [==============================] - 0s 665us/step - loss: 0.0662 - accuracy: 0.9831
Epoch 327/500
12/12 [==============================] - 0s 914us/step - loss: 0.0052 - accuracy: 1.0000
Epoch 328/500
12/12 [==============================] - 0s 997us/step - loss: 0.0029 - accuracy: 1.0000
Epoch 329/500
12/12 [==============================] - 0s 997us/step - loss: 0.0152 - accuracy: 1.0000
Epoch 330/500
12/12 [==============================] - 0s 914us/step - loss: 0.0382 - accuracy: 0.9831
Epoch 331/500
12/12 [==============================] - 0s 914us/step - loss: 0.0071 - a

Epoch 481/500
12/12 [==============================] - 0s 914us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 482/500
12/12 [==============================] - 0s 997us/step - loss: 0.0324 - accuracy: 0.9831
Epoch 483/500
12/12 [==============================] - 0s 997us/step - loss: 0.0433 - accuracy: 0.9831
Epoch 484/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0052 - accuracy: 1.0000
Epoch 485/500
12/12 [==============================] - 0s 998us/step - loss: 0.0161 - accuracy: 1.0000
Epoch 486/500
12/12 [==============================] - 0s 831us/step - loss: 0.0092 - accuracy: 1.0000
Epoch 487/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0105 - accuracy: 1.0000
Epoch 488/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 489/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 490/500
12/12 [==============================] - 0s 1ms/step - loss: 0.0055

In [16]:
train_y

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,

In [17]:
train_y_1 = []

In [39]:
train_x

[[1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0]

In [21]:
for i in range(0,len(train_y)):
    for j in range(0,15):
        if(train_y[i][j] == 1):
            train_y_1.append(j)

In [22]:
train_y_1

[13,
 8,
 6,
 11,
 11,
 7,
 8,
 5,
 10,
 1,
 2,
 8,
 14,
 12,
 5,
 12,
 11,
 0,
 11,
 6,
 6,
 10,
 10,
 3,
 7,
 13,
 9,
 10,
 13,
 0,
 0,
 7,
 5,
 7,
 4,
 6,
 14,
 3,
 7,
 13,
 6,
 13,
 7,
 2,
 12,
 14,
 10,
 7,
 1,
 0,
 2,
 14,
 10,
 4,
 3,
 9,
 12,
 9,
 4]

In [23]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()

In [31]:
classifier.fit(train_x,train_y_1)

GaussianNB()

In [32]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from tensorflow.keras.models import load_model
model = load_model('our_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [59]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = classifier1.predict(np.array([p]))[0]
    res1=[]
    for i in range(0,15):
        if(res == i):
            res1.append(1)
        else:
            res1.append(0)
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res1) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [60]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [63]:
chatbot_response("corona vaccine")

'You can book you vaccines from cowin website - visit this website www.cowin.gov.in'

In [64]:
from sklearn.linear_model import LogisticRegression
classifier1 = LogisticRegression(random_state = 0)
classifier1.fit(train_x, train_y_1)

LogisticRegression(random_state=0)

In [65]:
import pickle

In [66]:
with open('model_pickle','wb') as f:
    pickle.dump((classifier1),f)